# Dogs vs. Cats Redux: Kernels Edition
Distinguish images of dogs from cats 
https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/

using progressive resizing

In [14]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Load dependecies from fast.ai and numpy libraries.

In [15]:
from fastai.vision import *
from fastai.metrics import error_rate
import numpy as np

Set batch size to 4 to fit GPU requirements

In [16]:
bs = 4

In [17]:
path = Path('/notebooks/storage/data/dogs_cats_simple')
path_train = path/'train'
path_test = path/'test'

In [18]:
fnames = get_image_files(path_train)
fnames_test = get_image_files(path_test)

In [19]:
np.random.seed(2)
pat = r'/([^/]+)\.\d+.jpg$'

In [23]:
doc(ImageDataBunch.from_name_re)

## Stage 1 - Low Resolution

Start fitting the model with a lower resolution image with resnet34 then again with the same images at a higher resolution.

In [24]:
data = ImageDataBunch.from_name_re(path, fnames, pat, ds_tfms=get_transforms(), size=128, bs=bs, test=path_test
                                  ).normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)
learn.lr_find()

In [26]:
learn.fit_one_cycle(4, max_lr=slice(1e-5,1e-2))

epoch,train_loss,valid_loss,error_rate,time
0,0.516962,0.187748,0.073200,03:51
1,0.426745,1.160199,0.114600,03:51
2,0.328385,0.310579,0.064400,04:00
3,0.360245,0.216526,0.051800,03:56


In [ ]:
learn.recorder.plot()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(15,11))
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

## Stage 2 - High resolution

Increase the resolution of the images and continue fitting the model.

In [28]:
data = ImageDataBunch.from_name_re(path, fnames, pat, ds_tfms=get_transforms(), size=256, bs=bs, test=path_test
                                  ).normalize(imagenet_stats)
learn.data = data

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4, max_lr=slice(1e-5,1e-2))

In [ ]:
learn.save('stage-2-high')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(15,11))
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

## Format for kaggle submission

In [ ]:
classification = preds[:,1]
preds, y, losses = learn.get_preds(ds_type=DatasetType.Test, with_loss=True)
ids = np.array([int(f.stem) for f in fnames_test])
subm = np.stack([ids,classification], axis=1)
np.savetxt('submission.csv', subm, fmt='%d,%.5f', header='id,label', comments='')